## BLOG POST - CHI SQUARE 

*Please note I have chosen to use GitHub / Jupyter notebooks in substitute of Tumblr for my blog assignments

Upon analysis of the purpose of the loan, I notice that "debt consolition" has the highest percentage of defaults vs 
non-defaults coming in at 7.13%.  It is also important to note that this category contains the highest number of 
loans overall across all the groups.

Upon analyzing the default vs binned interest rates, I notice that the 10% - 15% group has the highest percentage of 
defaulted loans at 7.30%.  At the lowest and highest bins of the interest rates (0% - 10%, 20% - 25%), it actually has 
the lowest number percentage of loans in default.  The debt to income ratio bin follows the same pattern.  My original 
reserach question - do higher interest rates / debt to income ratios lend to higher defaults.  
For now the answer is no, higher interest rates and dti do NOT lend to higher defaults. 

In addition, for each individual test of the interest rate, the p-value is less 
than 0.05 to indicate that this test is significant and they are associated.

In [1]:
/* =================================
   1. DATA IMPORT (on local machine) 
   ================================ */
options nosource nonotes;

proc import datafile = '/folders/myfolders/sasuser.v94/LoanStats3a.csv'
out = work.loanstats
dbms = CSV;
run;

/* We can ingore error: ERROR: Expecting page 1, got page -1 instead. */

/* =======================
   2. DATA TRANSFORMATIONS 
   ======================= */

options nosource nonotes;

/* Had to put this all in a single step and assign to existing table - in order to append onto the table for charting. */
/* DATA new; set work.loanstats; */
DATA work.loanstats; set work.loanstats;

/* ==============================================================================
   Here I need to convert some data - int_rate = 25% - needs to be adjusted to .25  
   "_d" denotes decimal format */

/* All consolidated into one step */
int_rate_d = (input(substr(int_rate,1,index(int_rate, '%')-1), 8.) / 100);


/* same with revoling utilization */
revol_util_d = (input(  substr(revol_util,1,index(revol_util, '%')-1)  , 8.) / 100);


/* =================================
    Variable Adding / Binning */
    
/* If the recoveries or collection_recovery_fee > 0 then assign a default value of 1 */
IF recoveries > 0 THEN defaulted = 1;
ELSE IF collection_recovery_fee > 0 THEN defaulted = 1;
ELSE defaulted = 0;

/* int_rate */
length int_rate_bin $20; /* Set length */
IF int_rate_d < 0.10 THEN int_rate_bin = "0% - 10%";
ELSE IF int_rate_d >= 0.10 AND int_rate_d < 0.15 THEN int_rate_bin = "10% - 15%";
ELSE IF int_rate_d >= 0.15 AND int_rate_d < 0.20 THEN int_rate_bin = "15% - 20%";
ELSE IF int_rate_d >= 0.20 AND int_rate_d < 0.25 THEN int_rate_bin = "20% - 25%";
ELSE IF int_rate_d >= 0.25 AND int_rate_d < 0.30 THEN int_rate_bin = "25% - 30%";
ELSE IF int_rate_d >= 0.30 THEN int_rate_bin = "> 30%";
ELSE int_rate_bin = "N/A"; /* Catch bad data */

/* dti - range of 0-30 */
length dti_bin $20; /* Set length */
IF dti < 1 THEN dti_bin = "0 - 1";
ELSE IF dti >= 1 AND dti < 10 THEN dti_bin = "1 - 10";
ELSE IF dti >= 10 AND dti < 20 THEN dti_bin = "10 - 20";
ELSE IF dti >= 20 AND dti < 30 THEN dti_bin = "20 - 30";
ELSE IF dti >= 30 THEN dti_bin = "> 30";
ELSE dti_bin = "N/A"; 

/* revol_util */
length revol_util_bin $20; /* Set length */
IF revol_util_d < 0.25 THEN revol_util_bin = "0% - 25%";
ELSE IF revol_util_d >= 0.25 AND revol_util_d < 0.50 THEN revol_util_bin = "25% - 50%";
ELSE IF revol_util_d >= 0.50 AND revol_util_d < 0.75 THEN revol_util_bin = "50% - 75%";
ELSE IF revol_util_d >= 0.75 AND revol_util_d <= 1 THEN revol_util_bin = "75% - 100%";
ELSE IF revol_util_d > 1 THEN revol_util_bin = "> 100%";
ELSE revol_util_bin = "N/A"; 

/* =======================
    Set some labels */
LABEL   purpose = "Purpose for Loan"
        int_rate = "interest rate"
        dti = "debt to income ratio"
        revol_util = "revolving utilization"
        int_rate_d = "interest rate"
        revol_util_d = "revolving utilization"


        int_rate_bin = "interest rate (binned)"
        dti_bin = "debt to income ratio (binned)"
        revol_util_bin = "revolving utilization (binned)"
        defaulted = "Defulted Loan";
        
PROC SORT; by id;

/* Here split out a separate data set for the defaulted loans - for analysis use later */
DATA work.loanstats_defaulted; 
   SET work.loanstats;
   
    /* Sub set the data to only recoveries */
    IF defaulted = 1;
   
RUN;

## CHI SQUARE
<a id='chi_square'></a>

In [2]:
/* CHI SQ test on defaulted vs purpose */
proc freq data=work.loanstats; tables defaulted * purpose / CHISQ;
run;

In [3]:
/* CHI SQ test on defaulted vs int_rate_bin */
proc freq data=work.loanstats; tables defaulted * int_rate_bin / CHISQ;
run;

In [4]:
/* CHI SQ test on defaulted vs dti_bin */
proc freq data=work.loanstats; tables defaulted * dti_bin / CHISQ;
run;

In [5]:
/* run individually on interest rates 0-10 vs 10-15 */
data comparison1; set work.loanstats;
IF int_rate_bin = "0% - 10%" or int_rate_bin = "10% - 15%";
proc sort; by id;
proc freq; tables defaulted * int_rate_bin / CHISQ;
run;

In [6]:
/* run with a comparison of each category - 0-10 / 15-20  */
data comparison2; set work.loanstats;
IF int_rate_bin = "0% - 10%" or int_rate_bin = "15% - 20%";
proc sort; by id;
proc freq; tables defaulted * int_rate_bin / CHISQ;
run;

In [7]:
/* run with a comparison of each category - 0-10 / 15-20  */
data comparison3; set work.loanstats;
IF int_rate_bin = "0% - 10%" or int_rate_bin = "20% - 25%";
proc sort; by id;
proc freq; tables defaulted * int_rate_bin / CHISQ;
run;

In [8]:
/* HEre look at 15-20 and 20-25 */
data comparison4; set work.loanstats;
IF int_rate_bin = "15% - 20%" or int_rate_bin = "20% - 25%";
proc sort; by id;
proc freq; tables defaulted * int_rate_bin / CHISQ;
run;